# MICRO-452 Project Report - Groupe 22

**Authors:** David Croce, Leen Daher, Leonie Gasser, Jiasheng Wan                   

**Supervisors:** Prof. Francesco Mondada                    

**Due date:** 07.12.2023          

# Table of Contents


<ol>
<li>Introduction</li>
<li>Vision</li>
<li>Global Navigation</li>
<li>Local Navigation</li>
<li>Motion Control</li>
<li>Kalman Filter</li>
<li>Main</li>
<li>Conclusion</li>
</ol>

First we need to import all the necessary libraries and files:

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time 
import sys 

sys.path.append('.\src')

from main import *

ModuleNotFoundError: No module named 'vision'

In [ ]:
from tdmclient import ClientAsync
client = ClientAsync()
node = await client.wait_for_node()
await node.lock()

# 1. Introduction

In our project, Thymio is going on a walk in the forest next to the lake when he suddenly found himself surrounded by a fire. Luckily, he is able to communicate with his bird friends, who give him a top view of where the fire is, and how to reach the lake from his current position. On the way he avoids fallen trees, while his bird friends report any new fires that occur along his journey and help him replan his path. 

The simulation environment for testing consists of a white background, rectangular red obtsacles representing the flames the robot is attempting to avoid, white tuble-like obstacles representing the fallen trees, a rectangular blue goal representng the water the robot wants to reach, and four ArUco markers at each corner of the map. It is worth noting that the shape of the goal and the obstacles does not matter given that the detection algorithm is based on detecting the color. The markers allow us to divide the map into a grid to fascilitate path planning, given that we have settled on the A* algorithm. At some instances, since the smoke of the fire would block the birds' view, the designed Kalman filter will help the Thymio navigate.



The code consists of two main classes and two helper classes: 
### Main classes
 +  `ArucoMarker()` class in the python file vision.py discussed in the Vision section of the report.
 +  `Map()` class in the python env.py file discussed in the Global Navigation section of the report.
 +  `kalman()` class in the python file kalman.py discussed in the Kalman Filter section of the report.

 ### Helper classes
 +  `robot()` class in the python file robot.py
 +  `environment()` class in the python file env.py

The helper classes are used to organize the different attributes of the code like the map size and robot motion. The `robot()` class includes different functions to command to robot to move forward, turn, and stop, where each function is called based on inputs from different parts of the code. The main function of this class is shown in the cell below:



In [ ]:
def run_robot(self):
    """
    This function is used to control the robot. It is callaed in a thread and runs at 10HZ.
    """

    while True:
        
        if self.state != 'FINISH':
        
            if np.abs(self.teta) > 40:
                self.turn(0,self.teta)
                self.state = 'TURN'
            else:
                self.go_forward(self.teta)
                self.state = 'FORWARD'
            
        else:
            self.stop()
            break


While the `environment()` module helps to get the length and width of the map through the detected ArUco markers inorder to create the boundaries of the grid. 


# 2. Vision


The vision part of the project consists of two main blocks hat are the ArUco markers detection and tracking, in addition to color detection. The former is utilized in order to detect and assign the corners of the map and the position of the Thymio robot, while the latter works on detecting the the two colors red and blue. This is done using a camera that is positioned to have a top and full view of the map.

### ArUco Marker Detection and Tracking
As previously mentioned, this part of the code is responsible for detecting the ArUco markers placed on each corner of the map in additoin to the robot. ArUco markers are synthetics squares consisting of a binary matrix definig and identifier inside of a thick black border. Th detection of these markers is done using the aruco module of OpenCV, which is based on the ArUco library. The code provided below is part of the aruco.py pyhton file the group generated for the vision part of the project, and it includes the functions used to detect markers.


In [ ]:
def update_marker(self, frame):

    # Camera parameters
    focal_length = 1000  # Example focal length (adjust as needed)
    center = (frame.shape[1] / 2, frame.shape[0] / 2)  # Center of the frame

    # Camera matrix 
    cameraMatrix = np.array([[focal_length, 0, center[0]],
                                    [0, focal_length, center[1]],
                                [0, 0, 1]], dtype=np.float64)

    distCoeffs = np.zeros((4, 1))  
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    corners, ids, _ = cv2.aruco.detectMarkers(gray, self.aruco_dict, parameters=self.parameters)
    
    if ids is not None and self.marker_id in ids:
        idx = np.where(ids == self.marker_id)[0][0]
        rvecs, tvecs, _ = cv2.aruco.estimatePoseSingleMarkers(corners[idx], self.marker_size, cameraMatrix, distCoeffs)


        self.marker_pxl_size = np.linalg.norm(corners[idx][0][0] - corners[idx][0][1])

        axis_length = self.marker_size / 2
        points = np.float32([[0, 0, 0], [axis_length, 0, 0], [0, axis_length, 0], [0, 0, -axis_length]]).reshape(-1, 3, 1)
        axis_points, _ = cv2.projectPoints(points, rvecs[0], tvecs[0], cameraMatrix, distCoeffs)
        axis_points = axis_points.astype(int)

        angle = np.arctan2(axis_points[0].ravel()[0] - axis_points[1].ravel()[0], axis_points[0].ravel()[1] - axis_points[1].ravel()[1])
        angle = np.degrees(angle) + 90
        self.pos = axis_points[0].ravel()
        frame = cv2.circle(frame, tuple(self.pos), 3, (0, 255, 0), -1)

        # angle goes from 0 to 270 ang 0 to -90
        if(angle > 180):
            angle = angle - 360
        self.angle = angle

        if(self.marker_id == 1):
            frame = cv2.line(frame, tuple(axis_points[0].ravel()), tuple(axis_points[1].ravel()), (0, 255, 0), 2)
            self.camera_blocked = False
    
        if(self.marker_id == 4):

            #display all the unit positions
            for i in range(UNIT_NUMBER):
                for j in range(UNIT_NUMBER):
                    unit_pos = np.array([Map_camera[i][j][0], Map_camera[i][j][1]]) 
                    frame = cv2.circle(frame, (int(unit_pos[0]),int(unit_pos[1])), 1, (0, 255, 0), -1)
                    # Ensure unit_pos contains integer values and convert to tuple
                    unit_pos = tuple(map(int, unit_pos))
    else:
        self.camera_blocked = True

    return frame

The detection of the ArUco markers is done in the line: \
`    corners, ids, _ = cv2.aruco.detectMarkers(gray, self.aruco_dict, parameters=self.parameters)` \ 
where we extract the corners and the id of each detected marker. This information, in addition to the cameral parameteers are then used to estimate the pose of each detected marker and extracting their translation and rotaton vectors with respect to the camera.
These vectors are then utilized to find the 1 dimensional array that represents the flattened 2-D image of the coordinates of the projected points of the markers.

This method is used due to its ability to accurately identify the markers and estimate their pose with respect to the camera, which is useful in the case where we want to get the orientation and location of Thymio on the map.


source: https://docs.opencv.org/4.x/d5/dae/tutorial_aruco_detection.html

### Goal and Obstacle Detection
The same algorithm for detecting the obstacles and the goal is used, but wth different parameters to adjust for the different colors, where the color red implies a detected obstacle and the color blue implies a detected goal. The method is used over other edge detection methods since we can easiliy limit the colored objects in the map, but we can't limit the edges, and sharp objects to just the obstacles and the goal, and then there would be the issue of differentiating between the goal and the obstacles. Hence why the team settled on color detection. A sample of one of the codes is shown below for the blue color (goal) detection.


In [ ]:
def detect_goal(self,frame):
    # Convert the frame to HSV color space
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # Define the lower and upper bounds for the blue color in HSV
    lower_blue = np.array([90, 50, 50])
    upper_blue = np.array([130, 255, 255])
    # Create a mask to isolate blue regions in the image
    mask = cv2.inRange(hsv, lower_blue, upper_blue)

    # Apply morphological operations to remove noise
    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

    # Find contours of blue objects
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    self.detected_goal = []
    # Loop through the contours to find the bounding boxes of blue objects
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > 500:  # Set a minimum area threshold
            x, y, w, h = cv2.boundingRect(contour)
            self.centroid_goal = np.array([x + w/2, y + h/2])

            if w > 30 and h > 30:  # Set minimum width and height thresholds
                top_left = (x, y)
                bottom_right = (x + w, y + h)
                self.detected_goal.append({'top_left': top_left, 'bottom_right': bottom_right})
                
                # Draw contours around the detected objects on the frame
                cv2.rectangle(frame, top_left, bottom_right, (200, 0, 0), 2)

    return frame

The color detection algorithm first converts the frame from RGB(Red- Green - Blue) to HSV (Hue-Saturaton-Value). We then define the range of our blue detectr to create a mask. `cv2.inRange()` method is used for thresholding. The output of this function will be a binary mask consisting of black and white pixels, where the white pixels indicate the colors in the specified threshold, and the black pixels indicate the colors that fall outside that range.\
After generating the mask, morphological transformations are applied to the binary image to reduce noise. This is done using `cv2.morphologyEX()`, where the function does erosion followed by dilation of the picture, or dilation followed by erosion, depending whether the input is `cv.MORPH_OPEN` or `cv.MORPH_CLOSE` respectively. Erosion is when pixels around the boundary of the white objects are discarded depending on the kernel size, while dilation is the opposite of erosion, where it increases the wite region around the edges of the white pixels of the image. \
Finally, the function `cv2.findContours()` is used to identify the shape provided in the binary image, and it outputs the coordinates of the coordinates of the four corners of the object.


source 1:https://docs.opencv.org/3.4/d9/d61/tutorial_py_morphological_ops.html

### Generating the Map
After assigning the for corners of the map, and the positions of the goal and the obstacles, we now work on transforming it into a square grid to be able to implement the path planner on it. The code below explains the generation of the grid map:



In [ ]:
Map_camera = np.zeros((UNIT_NUMBER,UNIT_NUMBER,2)) # 2D array containing the position of each unit in the map
Map_indices = np.zeros((UNIT_NUMBER,UNIT_NUMBER))

def update_map_matrix(self,frame):
    # Matrix representing the map. To be filled and used for path planning
    matrix = np.zeros((UNIT_NUMBER,UNIT_NUMBER))

    OBSTACLE_MARGIN = 0

    # Set to 1 the units where the obstacles are
    for i in range(self.nb_obstacles):
        top_left = self.detected_obstacles[i]['top_left']
        bottom_right = self.detected_obstacles[i]['bottom_right']

        for j in range(UNIT_NUMBER):
            for k in range(UNIT_NUMBER):
                if top_left[0] - OBSTACLE_MARGIN < Map_camera[j][k][0] < bottom_right[0] + OBSTACLE_MARGIN and top_left[1] - OBSTACLE_MARGIN < Map_camera[j][k][1] < bottom_right[1] + OBSTACLE_MARGIN:
                    matrix[UNIT_NUMBER-k-1][j] = OBSTACLE
                    frame = cv2.circle(frame, (int(Map_camera[j][k][0]),int(Map_camera[j][k][1])), 1, (0, 0, 255), -1)
    ct_gl = 0
    for j in range(UNIT_NUMBER):
        for k in range(UNIT_NUMBER):
            if self.centroid_goal[0] - PIXEL_MARGIN < Map_camera[j][k][0] < self.centroid_goal[0] + PIXEL_MARGIN and self.centroid_goal[1] - PIXEL_MARGIN < Map_camera[j][k][1] < self.centroid_goal[1] + PIXEL_MARGIN:
                
                ct_gl = ct_gl + 1
                if ct_gl == 1:
                    self.goal_idx = np.array([j, k])
                    frame = cv2.circle(frame, (int(Map_camera[j][k][0]),int(Map_camera[j][k][1])), 3, (0, 0, 255), -1)
                break

    
    if self.marker_id == 1:
        #create array with all the matrix units corresponding to robot
        ct = 0
        for j in range(UNIT_NUMBER):
            for k in range(UNIT_NUMBER):
                if self.pos[0] - PIXEL_MARGIN < Map_camera[j][k][0] < self.pos[0] + PIXEL_MARGIN and self.pos[1] - PIXEL_MARGIN < Map_camera[j][k][1] < self.pos[1] + PIXEL_MARGIN:
                    
                    ct = ct + 1
                    if ct == 1:
                        frame = cv2.circle(frame, (int(Map_camera[j][k][0]),int(Map_camera[j][k][1])), 3, (255, 0, 0), -1)
                        self.robot_idx = np.array([j, k])
                
                    break
    
    
    self.Map_indices = matrix

In this code we first create an empty matrix of a desired size that will then be filled with the positions of the obstacles. We then loop over the detected number of obstacles, that is given by the `detect_red_objects()` function of the ArucoMarker class, and extract the required coordinates of the obstacle. Then, each point of the map is checked to see whether an obstacle passes through it or not, and incase the statement is true, the element of the matrix that represents this point on the camera is set to 1 indicating that this point cannot be accessed by the planner. This code is also used to pinpoint the position of the robot and the goal on the grid. 

# 3. Global Navigation



For the global navigation, the camera creates a grid-based map, locating Thymio as well as obstacles and the goal as mentioned in the previous parts. Then the path is planned based on the `find_shortest_path()` function in the `astar_path_planning.py` python file, which is based on the A* algorithm. The choice of A* was made in order to frequently update the map as it is computationally less expensive than Dijkstra's algorythm while still compatible with our grid based map and finding the shortest path.

The first iteration of the planner had the limitation of getting stuck in local minimum. This occured when obstacles were too close to one another, or they were touching the boarders of the map. Given that our map is not big, and taking into account the size of the Thymio, our planner woud frequently get stuck in local minmum, and hence required either to change the planner or improve it. So, the team chose to build on the A* algorithm by removing the heuristic function, which led to a planner more reselmbling a breadth first sarch (BFS). This meant that we had to compromise the optimality of our path over its completeness, in order to have a complete path planner. Once we realised that the results were no longer optimal, we modified our now reliable funciton to add the heuristic function back in. This results in our current planner which is a modified A*.

The function `is_valid()` is then implemented to to take into account Thymio's dimensions with respect to the obstacles. The function checks wether or not a potential node in the path is a valid option for Thymio to go to or if it is too close to an obstacle. After calculating the path, it is then broken down into the coordinates where the direction is changed and this infromation is then sent to motion control. Having the individual path points as far appart as possible facilitates the angle calculation between the Thymio and the next path point, which is needed for the motion control. 




# 4. Local Navigation



Our local navigation is a simple one layer neural network with the infrared sensors as the weighed inputs and the motor speed as outputs. The weights were determined using trial and errror, such that if an obstacle is detected, Thymio turns away from it and drives a short distance, before it aligns itself again with the next path point to check if the obstacle is still in the way. As soon as the obstacle is no longer detected by the infrared sensors, it goes back to following the path. Which insures that the it doesnt get distracted with circleing the obstacle.



# 5. Motion Control


The motion control strategy is based on the path points as mentioned above. The main idea being that Thymio turns on the spot until it faces the next point of the path and then drives in a straight line untill it reaches it. In order for this to be a smooth process there is a threshhold where if the angle is under 40°, Thymio drives and corrects the angle at the same time. 

The motion controller employs a proportional (P) controller to maintain Thymio on the correct path. The Thymio's speed is calculated as the average of the velocities of the left and right wheels.

$$ \text{speed} = \frac{W_l + W_r}{2} $$

The rotation rate $\dot{\theta}$ of the Thymio is determined by calculating the difference in speed between the wheels and dividing it by the distance between the two wheels.

$$ \dot{\theta} = \frac{W_r - W_l}{L} $$

$\dot{\theta}$: Rotation rate \
$W_r$: Velocity of right wheel \
$W_l$: Velocity of left wheel \
$L$: Distance between the two wheels


# 6. Position Estimation: Kalman Filter



Accurate localization of the Thymio is fundamental for successful navigation and obstacle avoidance. The Kalman filter was employed to determine and update the Thymio's position and orientation.

The state of the Thymio's position and orientation was updated using the Kalman filter. At time t-1, the Thymio had the current state with the following state vector: 

$$ 
x_{t} = \begin{bmatrix} x_{t} \\ y_{t} \\ \theta_{t} \end{bmatrix}
$$

  $x$: $x$ coordinate position of the Thymio \
  $y$: $y$ coordinate position of the Thymio \
  $\theta$: Angle or orientation of Thymio 

In the state space, the kalman filter maintains the estimate of the x, y and $\theta$ values. It continuously updates the estimates based on sensor measurement and motion model prediciton using the kalman filter. The linear state space model of the 2 wheel mobile robot is given by:

$$
\begin{bmatrix} x_{t} \\ y_{t} \\ \theta_{t} \end{bmatrix} = A_{t-1} \cdot \begin{bmatrix} x_{t-1} \\ y_{t-1} \\ \theta_{t-1} \end{bmatrix} + B_{t-1} \cdot \begin{bmatrix} v_{t-1} \\ \omega_{t-1} \end{bmatrix}  +  w_{t}
$$

$$
y_{t} = H \cdot x_{t} + \nu_{t}
$$

where 

$$
A_{t-1} = 
\begin{bmatrix} 1 && 0 && 0 \\ 0 && 1 && 0 \\ 0 && 0 && 1 \\ \end{bmatrix}
$$
$$
B_{t-1} = 
\begin{bmatrix} cos_{\theta_{t-1}} &&  0 \\ sin_{\theta_{t-1}} && 0 \\ 0 && dt \\ \end{bmatrix}
$$

  $H$: Measurement matrix
  $v_{t-1}$: Velocity of Thymio \
  $\omega_{t-1}$: Angular velocity of Thymio \
  $w_{t}$: Process noise \
  $\nu_{t}$: Measurement noise

The dynamics of the Thymio is caputured by the transition matrix A and control input B into the Kalman filter. This dynamics of system evolves over the inerations when the Thymio moves.

The covariance prediction is used during the prediction step to update the covariance matrix. It represents the uncertainty of the state estimate and it can be written as:

$$ P_{t+1} = A \cdot P_{t} \cdot A^T + Q $$


$P_\text{priori}$ : Covariance matrix of the next state estimation \
$P$ : Covariance matrix of the current state estimation \
$Q$ : Process noise

Assuming uncorrelated process noise (Q) in the x axis, y axis and $\theta$, the mutual terms can be set to zero.

H is the measurement matrix, also an identity matrix, that maps the state space to the measurement space. The covariance matrix, denoted as R in the script, captures the uncertainty in the measurement. The measurement in our case is the x position and y position obtained from the camera. The value R will be used in the update step to adjust the Kalman gain.

$$
K_t = P_{t-1} \cdot H^T \cdot (H \cdot P_{t-1} \cdot H^T + R)^{-1}
$$

$K_t$ : Kalman gain at time t \
$P_t^-$: Predicted (priori) estimate covariance matrix \
$R$: Measurement noise

When the camera is obstructed, it results in a noisy measurement. The R value is set to infinity, indicating a high level of uncertainty in the sensor measurement. Consequently, the Kalman filter assigns less weight to individual measurements to mitigate the impact of noise, relying more on the motion model. This adaptation makes the Kalman filter less responsive to camera measurements.

Innovation refers to the disparity between measured data and predicted data. It represents the error in the prediction and plays a pivotal role in the update step of the Kalman filter.

$$
\text{Innovation} = y_{t} - H \cdot x_{t-1}
$$

The Kalman gain and innovation obtained will be utilized in the update step of the state model. The state estimate is a fusion of the predicted motion model and the actual measurements

$$
x_t = x_{t-1} + K_t \cdot Innovation
$$

During the Kalman filter update, the obtained estimate $x_{t}$ is a vector that contains the x position, y position and orientation $\theta$ of the Thymio. The obtained $x_{t}$ will be used to refine the estimate of the current state of the system.

After designing the filter, a short simulating with no obstacles on the map is run to check whether the robot is able to estimate the future positions even in the absence of the camera. The results of the test are given in Figure 1, where the constant values of the camera position reading indicate that the camera is turned off and the kalman filter is activated. The blue point shows the estimate of the position of the robot. The plot indicates that the filter is able to adequately estimate the future positions of the Thymio in the absence of the camera when the robot is driving in the forward direction with a small error (~10 pixels).  
![Figure 1. Position estimation along the x-direction when the camera is off](kalman_plot_x.png)



Source 1: https://automaticaddison.com/how-to-derive-the-state-space-model-for-a-mobile-robot/
Source 2: Becker, A. (2023). Kilman Filter: From the ground up. KilmanFilter.NET. 




# 7. Main

This section of the report includes the main function where all the modules are interacting with each other. In this folder, Here, our only output is the speed sent to each motor, where the speeds are generated by calling `update_main()` in the main.py python file inside an infinite loop in order to find the required motor speeds.

In [ ]:

def motors(left, right):
    return {
        "motor.left.target": [left],
        "motor.right.target": [right],
    }

def get_data():
    thymio_data = np.array({"left_speed":node["motor.left.speed"],
                        "right_speed":node["motor.right.speed"]})

    return thymio_data

start_time = time.time()
while True:
    

    await node.wait_for_variables({"motor.left.speed", "motor.right.speed"})
    speed_left = node.v.motor.left.speed/2
    speed_right = node.v.motor.right.speed/2
    
    avg_speed = (speed_left+speed_right)/2

    angular_speed = (speed_right-speed_left)/80
    #print(node.v.motor.left.speed, "   ", node.v.motor.right.speed)

    v = update_main(avg_speed, angular_speed)
    node.send_set_variables(v)
    await client.sleep(0.1)



In [ ]:
def motors(left, right):
    return {
        "motor.left.target": [left],
        "motor.right.target": [right],
    }

node.send_set_variables(motors(0,0))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time 
import sys 

from tdmclient import ClientAsync
client = ClientAsync()
node = await client.wait_for_node()
await node.lock()



def get_data():
    thymio_data = np.array({"left_speed":node["motor.left.speed"],
                        "right_speed":node["motor.right.speed"]})

    return thymio_data

v = {"motor.left.target": [100],
    "motor.right.target": [100],}


while True:
    await node.wait_for_variables({"motor.left.speed", "motor.right.speed"})
    print(node.v.motor.left.speed, "   ", node.v.motor.right.speed)
    
    node.send_set_variables(v)

    await client.sleep(0.1)
 

The code snippet below is used to show a simulation of how the path planner geneerates the pat based on a given input grid matrix provided to it by the camera. The grid is a matrix consisting of 0 and 1 elements where 0 corresponsds to free space, and 1 corresponds to a space occupied by and obstacle.

In [ ]:
import matplotlib.pyplot as plt
from collections import deque
import numpy as np

def is_valid(x, y, grid, robot_size):
    n = len(grid)
    for i in range(max(0, x - robot_size + 1), min(n, x + robot_size)):
        for j in range(max(0, y - robot_size + 1), min(n, y + robot_size)):
            if not (0 <= i < n and 0 <= j < n) or grid[i][j] == 1:
                return False
    return True

from collections import deque

def heuristic(cell, goal):
    # A simple heuristic function (Euclidean distance)
    return ((cell[0] - goal[0]) ** 2 + (cell[1] - goal[1]) ** 2) ** 0.5

def find_shortest_path_Astar(grid, start, goal, robot_size):
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1), (-1, -1), (-1, 1), (1, -1), (1, 1)]
    queue = deque([(start, [],0)])
    visited = set()
    n = len(grid)

    while queue:
        (x, y), path, cost = queue.popleft()
        if (x, y) == goal:
            return path + [(x, y)]

        if (x, y) not in visited and 0 <= x < n and 0 <= y < n:
            visited.add((x, y))
            for dx, dy in directions:
                new_x, new_y = x + dx, y + dy
                if is_valid(new_x, new_y, grid, robot_size) and 0 <= new_x < n and 0 <= new_y < n:
                    new_path = path + [(x, y)]
                    cost = len(new_path) + heuristic((new_x, new_y), goal)
                    queue.append(((new_x, new_y), new_path, cost))

        # Sort the queue based on the total cost (g(n) + h(n))
        queue = deque(sorted(queue, key=lambda x: x[2]))

    return None



def plot_grid(grid, path, start, goal):
    n = len(grid)
    plt.figure(figsize=(8, 8))
    plt.imshow(grid, cmap='Greys', interpolation='nearest')

    if path:
        path_x, path_y = zip(*path)
        plt.plot(path_y, path_x, marker='o', color='red')

    plt.plot(start[1], start[0], marker='o', color='green', markersize=10)
    plt.plot(goal[1], goal[0], marker='o', color='blue', markersize=10)

    plt.xlim(-0.5, n - 0.5)
    plt.ylim(n - 0.5, -0.5)
    plt.xticks(np.arange(-0.5, n, 1))
    plt.yticks(np.arange(-0.5, n, 1))
    plt.gca().invert_yaxis()
    plt.grid(visible=True, color='black', linestyle='-', linewidth=0.5)
    plt.show()

def find_direction_changes(path):
    if len(path) <= 2:
        return path

    direction_changes = [path[0]]

    for i in range(2, len(path)):
        dx1 = path[i - 1][0] - path[i - 2][0]
        dy1 = path[i - 1][1] - path[i - 2][1]
        dx2 = path[i][0] - path[i - 1][0]
        dy2 = path[i][1] - path[i - 1][1]

        if (dx1, dy1) != (dx2, dy2):
            direction_changes.append(path[i - 1])

    direction_changes.append(path[-1])

    return direction_changes

# Example grid with obstacles

grid = np.array([
    [0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0],
    [0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0],
    [0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0],
    [0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0],
    [1 ,1 ,1 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,1 ,1],
    [1 ,1 ,1 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,1 ,1],
    [1 ,1 ,1 ,0 ,0 ,0 ,1 ,1 ,1 ,1 ,1 ,1 ,0 ,1 ,1],
    [0 ,0 ,0 ,0 ,0 ,0 ,1 ,1 ,1 ,1 ,1 ,1 ,0 ,1 ,1],
    [0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,1 ,1],
    [0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,1 ,1],
    [0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0],
    [0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0],
    [0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0],
    [0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0],
    [0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0]
    ])

#filp rows of grid
grid = np.flip(grid,0)

start_index = (2, 1)  # Replace with the index of your robot
goal_index = (12, 13)  # Replace with the index of your goal
robot_size = 2  # Replace with the size of your robot

path = find_shortest_path_Astar(grid, start_index, goal_index, robot_size)
if path:
    direction_changes_path = find_direction_changes(path)
    print("Path with direction changes:", direction_changes_path)
    plot_grid(grid, direction_changes_path, start_index, goal_index)
    # plot_grid(grid, path, start_index, goal_index)
else:
    print("No path found.")


# 8. Conclusion

In summary, the team worked on designing a map that would be detected using a camera placed above the map. First, the vision module is called to detect the main parts of the map consisting of the fire obstacles, the water goal, and the current position of the Thymio. This module is being called at every frame in order to keep updating the position of the robot. Then a grid-based map is generated from the inputs of the camera and fed to a modified A* global path planner to generate a viable path for the robot to follow. The output of this path is passed to the motion planner to orient and direct Thymio towards his next point in the map until he finally reaches his goal. A Kalman filter module is also implemented in the event that we lose view from the camera so the robot will be able to estimate its future state based on its last known state and still manage to proceed on its path. Finally, in the case of a sudden obstacle popping up infront of the robot, a local path planner is designd to help Thymio avoid it using its own sensors rather than depending on inputs from the camera. The teting of the system showed that the different modules are able to communicate well with each other in order to generate a path to avoid the different obstacles and reach  the goal point effectively. 